# RHF 与 RMP2 能量

在这一节中，我们会使用 PySCF 的库进行 RHF 与 RMP2 能量的计算，同时对其中的部分中间矩阵进行输出与性质的考察．通过这一节，我们应当可以对 Post-HF 中最为基础的向量、张量的导出与计算有所了解．

## 修改库函数的准备

PySCF 绝大部分与量子化学方法本身有关的内容使用 Python 编程，因此我们可以更改库函数中 PySCF 库的代码，直接地对库中的函数与变量进行调试．

但是，Jupyter Notebook 在默认情况下，当内核准备完毕后，对库的 `.py` 文件的更改不会反映在程序的更改上；除非重启内核．这应当是因为 Python 在进行 `import` 命令时，会预编译被引入的 `.py` 文件为 `.pyc` 文件，从而只读取二进制的 `.pyc` 文件即可以高效地执行程序；而避免从 `.py` 文件先读取未编译的代码，再经过 Python 解释器编译．

事实上，Jupyter Notebook 在加入下述 Magic Command 后，可以在同一个内核进程中，对库函数的更改作出响应．但想必因为要进行库函数的 `.py` 文件进行额外的编译操作，因此执行效率多少会慢一些．

In [ ]:
%load_ext autoreload
%autoreload 2

<div class="alert alert-warning">

**警告**

修改库程序始终是危险操作！在修改之前至少需要作一个备份！

</div>

<div class="alert alert-info">

**提示**

一般库函数的位置在 Anaconda 安装文件夹下 `lib\python3.*\site-packages\pyscf` 中．

</div>

## 顶层函数计算 RHF 与 RMP2

引入库函数时，只需要 PySCF 与 NumPy 库即可；但 PySCF 的库经常需要手动引入．这些库需要通过查看源代码或者文档来了解．

In [ ]:
from pyscf import gto, scf, mp, ao2mo, lib
import numpy as np

构建分子可以通过下述命令进行：

In [ ]:
mol = gto.Mole()
mol.atom = """
O  1.0  0.0  0.0
H  1.0  1.0  0.0
H  1.0  0.0  1.0
"""
mol.basis = "6-31G"
mol.build()

RHF 能量计算可以通过下述两行命令给出：

In [ ]:
scf_eng = scf.RHF(mol)
scf_eng.conv_tol = 1e-13
energy_RHF = scf_eng.kernel()

RMP2 能量计算也可以通过两行命令给出：

In [ ]:
mp2_eng = mp.MP2(scf_eng)
energy_RMP2_corr, _ = mp2_eng.kernel()

这些结果应当能与 Gaussian 进行比对，误差在第七位小数左右．Gaussian 的输入卡可以是：

```
#p MP2(Full)/6-31G

H2O

0 1
O  1.0  0.0  0.0
H  1.0  1.0  0.0
H  1.0  0.0  1.0
```

## HF 中间矩阵与变量

在 Post-HF 计算与程序编写过程中，我们会经常使用到 HF 中间矩阵．在这里我们作简单的介绍．

### 系数矩阵 $C_{\mu p}$

In [ ]:
C = scf_eng.mo_coeff

### 本征向量 $\varepsilon_p$

In [ ]:
e = scf_eng.mo_energy

### 轨道数与电子数

以下代码依次定义原子轨道数 $n_\mathrm{AO}$、分子轨道数 $n_\mathrm{MO}$、电子数 $n_\mathrm{elec}$、占据轨道数 $n_\mathrm{occ}$、未占轨道数 $n_\mathrm{vir}$．

In [ ]:
nao = mol.nao
nmo = scf_eng.mo_energy.shape[0]
nelec = mol.nelectron
nocc = mol.nelec[0]
nvir = nmo - nocc

由于我们已经假设了闭壳层以及没有轨道冻结，同时没有强基组线性依赖，因此以下的关系成立：

* $n_\mathrm{AO} = n_\mathrm{MO}$

In [ ]:
nao == nmo

* $n_\mathrm{elec} = 2 n_\mathrm{occ}$

In [ ]:
nelec == 2 * nocc

### AO 基组密度 $D_{\mu \nu}$

AO 基组密度可以通过下述公式获得：

\begin{equation}
D_{\mu \nu} = C_{\mu i} D_{ij} C_{\nu j}
\end{equation}

其中，$D_{ij} = 2 \delta_{ij}$ 是 MO 基组密度矩阵．注意到这里使用了 RHF 条件，即 $\alpha$ 与 $\beta$ 轨道密度相等，因此这里所使用的密度可能与不少程序的应用或教科书的定义相差两倍．

在 PySCF 中，可以使用 `scf.make_rdm1` 导出 AO 基组密度．我们可以验证这两种方法所导出的密度是相等的．

In [ ]:
D = scf_eng.make_rdm1()
np.allclose(C[:, :nocc] @ (2 * np.eye(nocc)) @ C[:, :nocc].T, D)

### AO 基组 Fock 矩阵 $F_{\mu \nu}$

这里我们仅仅写出 Fock 矩阵的导出方式．我们可以在对 AO 基组电子积分作简单介绍后，验证下述 Fock 矩阵的合理性．现在我们可以验证的，是 AO 基组 Fock 矩阵与轨道能之间的对应关系：

\begin{equation}
C_{\mu p} F_{\mu \nu} C_{\nu q} = \delta_{pq} \varepsilon_{p}
\end{equation}

In [ ]:
F = scf_eng.get_fock()
np.allclose(C.T @ F @ C, np.eye(nmo) * e)

<div class="alert alert-info">

**提示**

上述的验证在默认的 SCF 阈值 $10^{-10}$ 下，可能会给出 False 的判断．这是为何 [执行 SCF 计算](#顶层函数计算-RHF-与-RMP2) 时需要额外指定 SCF 收敛阈值的原因．

</div>

## 电子积分与导出矩阵

### 原子轨道积分矩阵 $S_{\mu \nu}$

原子轨道定义为

\begin{equation}
S_{\mu \nu} = \langle \mu | \nu \rangle = \int \phi_\mu (\boldsymbol{r}) \phi_\nu (\boldsymbol{r}) \, \mathrm{d} \boldsymbol{r}
\end{equation}

我们可以通过下式验证以下原子轨道积分：

\begin{equation}
C_{\mu i} S_{\mu \nu} C_{\nu j} = \delta_{ij}
\end{equation}

后文多会使用 Dirac 记号；被积变量 $\boldsymbol{r}$ 通常不会写出．

In [ ]:
S = scf_eng.get_ovlp()
np.allclose(C.T @ S @ C, np.eye(nmo))

在 PySCF 中，事实上除了使用 SCF 对象 `scf.hf.RHF` 外，还可以直接通过 `gto.mole.Mole` 中的方法直接给出电子积分．一般来说，使用 `gto.mole.Mole` 对象通常可以利用更为底层的函数获得积分，因此使用上较为灵活，同时代码并不比高层函数来得大很多．我们下面就可以通过 `gto.mole.Mole` 给出电子积分．更多的电子积分名称可以在 [PySCF 文档](https://sunqm.github.io/pyscf/gto.html#module-pyscf.gto.moleintor) 中找到．

In [ ]:
np.allclose(scf_eng.get_ovlp(), mol.intor('int1e_ovlp_sph'))

### 单电子积分 $T_{\mu \nu}, V_{\mu \nu}^\mathrm{Nuc}$

动能积分定义为

\begin{equation}
T_{\mu \nu} = \frac{1}{2} \langle \nabla^2 \mu | \nu \rangle
\end{equation}

其中，$\nabla$ 是在坐标空间上的三个维度上的梯度算符，其平方是指向量点积 $\nabla \cdot \nabla$．

势能积分定义为

\begin{equation}
V_{\mu \nu}^\mathrm{Nuc} = \langle \mu | \hat V_\mathrm{Nuc} | \nu \rangle = \langle \mu | \frac{Z_A}{| \boldsymbol{R}_A - \boldsymbol{r} |} | \nu \rangle
\end{equation}

在 PySCF 中，还可以使用 `get_hcore` 方法获得这两个单电子积分的和．我们可以验证其结果．

In [ ]:
T = mol.intor('int1e_kin_sph')
Vnuc = mol.intor('int1e_nuc_sph')
np.allclose(scf_eng.get_hcore(), T + Vnuc)

### 双电子 ERI 积分 $(\mu \nu | \kappa \lambda)$

双电子积分 ERI (Electron Repulsion Integral) 需要通过 `ao2mo` 进行计算．其定义为

\begin{equation}
( \mu \nu | \kappa \lambda ) = \int \phi_\mu (\boldsymbol{r}_1) \phi_\nu (\boldsymbol{r}_1) \frac{1}{|\boldsymbol{r}_1 - \boldsymbol{r}_2|} \phi_\kappa (\boldsymbol{r}_2) \phi_\lambda (\boldsymbol{r}_2) \, \mathrm{d} \boldsymbol{r}_1 \, \mathrm{d} \boldsymbol{r}_2
\end{equation}

由于这是原子轨道下的积分，因此公式中出现的轨道顺序是按照化学的约定俗成来排列．但在分子轨道下，不同的文献会使用不同的约定俗成．在以后的教程中，通常会使用物理的约定俗成来记分子轨道 ERI．

<div class="alert alert-info">

**提示**

双电子积分可能会占很大的内存．如果基组或分子较大，可能需要在调用积分函数前确保内存大小．

</div>

In [ ]:
eri = mol.intor('int2e_sph')
eri.shape

### 库伦积分 $J_{\mu \nu} [\mathbf{D}]$ 与交换积分 $K_{\mu \nu} [\mathbf{D}]$

在 SCF 中，库伦积分经常会定义为 (Szabo, 2.182, 2.184)

\begin{align}
J_{ij} &= \langle ij | ij \rangle \\
K_{ij} &= \langle ij | ji \rangle
\end{align}

随后，还可以将这些矩阵转换到原子轨道基组．

但在实际程序中，我们会作更宽泛的定义：

\begin{align}
J_{\mu \nu} [\mathbf{D}] &= (\mu \nu | \kappa \lambda) D_{\kappa \lambda} \\
K_{\mu \nu} [\mathbf{D}] &= (\mu \kappa | \nu \lambda) D_{\kappa \lambda}
\end{align}

当代入的密度不同，其库伦与交换积分的结果也会不同．PySCF 中也提供了计算库伦与交换积分的方法 `get_j` 与 `get_k`，我们可以用上面的公式验证．

In [ ]:
J = scf_eng.get_j()
np.allclose(np.einsum("uvkl, kl -> uv", eri, D), J)

In [ ]:
K = scf_eng.get_k()
np.allclose(np.einsum("ukvl, kl -> uv", eri, D), K)

### Fock 矩阵验证

在 RHF 下，Fock 矩阵可以表示为

\begin{equation}
F_{\mu \nu} = T_{\mu \nu} + V_{\mu \nu}^\mathrm{Nuc} + J_{\mu \nu} [\mathbf{D}] - \frac{1}{2} K_{\mu \nu} [\mathbf{D}]
\end{equation}

下面我们就可以验证这个结果．

In [ ]:
np.allclose(T + Vnuc + J - 0.5 * K, F)

<div class="alert alert-info">

**提示**

可能许多程序中，库伦积分与交换积分的倍数分别是 2 与 1；但这里由于自洽场密度取的是 $\alpha$ 与 $\beta$ 自旋密度的加和，因此这里的倍数都会少 2 倍．

</div>

<div class="alert alert-info">

**提示**

与代入密度相关的方法，譬如 `get_j`、`get_k`、`get_fock` 等，它们尽管默认了密度为自洽场密度 $D_{\mu \nu}$，但仍然可以代入其它任何的密度．这将会在以后的教程中使用到．

</div>

### SCF 能量验证

最后，我们可以验证最终的 Hartree-Fock 能量值：

\begin{equation}
E^\textsf{HF} = D_{\mu \nu} (T_{\mu} + V_{\mu}^\mathrm{Nuc} + \frac{1}{2} J_{\mu \nu} [\mathbf{D}] - \frac{1}{4} K_{\mu \nu} [\mathbf{D}]) +  E^\mathrm{Nuc}
\end{equation}

In [ ]:
np.allclose((D * (T + Vnuc + 0.5 * J - 0.25 * K)).sum() + mol.energy_nuc(), \
            scf_eng.energy_tot())

其中，$E^\mathrm{Nuc}$ 为核坐标排斥能：(这里暂不使用 Einstein Convention)

\begin{equation}
E^\mathrm{Nuc} = \sum_{A \neq B} \frac{Z_A Z_B}{| \boldsymbol{R}_A - \boldsymbol{R}_B |}
\end{equation}

通过下述的代码块可以验证该结果．

In [ ]:
Enuc_my = 0.
for A in range(mol.natm):
    for B in range(A + 1, mol.natm):
        Enuc_my += mol.atom_charge(A) * mol.atom_charge(B) / np.linalg.norm(mol.atom_coord(A) - mol.atom_coord(B))
np.allclose(scf_eng.energy_nuc(), Enuc_my)

## MP2 能量与中间张量

### MO ERI 积分 $g_{pq}^{rs}$

分子轨道基组的 ERI 积分定义为

\begin{equation}
g_{pq}^{rs} = \langle pq | rs \rangle = C_{\mu p} C_{\nu r} ( \mu \nu | \kappa \lambda ) C_{\kappa q} C_{\lambda s}
\end{equation}

在 PySCF 中，我们可以使用 `ao2mo` 对 AO 基组的 ERI 积分进行转换．默认情况下，转换后的轨道顺序是化学约定的；我们可以对换其中的第二、三根轨道顺序以得到物理约定的轨道顺序．下面就对这种方法获得的轨道进行验证．

In [ ]:
g_pqrs = ao2mo.general(mol, (C, C, C, C), compact=False).reshape((nmo, nmo, nmo, nmo)).swapaxes(1,2)
np.allclose(np.einsum("up, vr, uvkl, kq, ls -> pqrs", C, C, eri, C, C, optimize=True), g_pqrs)

在 MP2 计算中，实际上不需要使用全分子轨道的 ERI 张量，只需要提取其中占据-占据-非占-非占的部分 $g_{ij}^{ab}$ 即可．用相同的方法，我们可以获得该张量并进行验证：

In [ ]:
# 定义占据轨道系数 Co，未占轨道系数 Cv
Co = C[:, :nocc]
Cv = C[:, nocc:]

# 根据公式验证 oo-vv 部分的 MO ERI
g_ijab = ao2mo.general(mol, (Co, Cv, Co, Cv), compact=False).reshape((nocc, nvir, nocc, nvir)).swapaxes(1,2)
print(np.allclose(np.einsum("ui, va, uvkl, kj, lb -> ijab", Co, Cv, eri, Co, Cv, optimize=True), g_ijab))

# 验证 g_pqrs 所提取出的 oo-vv 部分与 g_ijab 相同
print(np.allclose(g_pqrs[:nocc, :nocc, nocc:, nocc:], g_ijab))

### 反对称 MO ERI $\bar g_{pq}^{rs}$

反对称 MO ERI 定义为

\begin{equation}
\bar g_{pq}^{rs} = \langle pq \Vert rs \rangle = g_{pq}^{rs} - g_{pq}^{sr}
\end{equation}

我们可以验证

\begin{equation}
\bar g_{pq}^{rs} = - \bar g_{pq}^{sr}
\end{equation}

In [ ]:
gbar_pqrs = g_pqrs - g_pqrs.transpose(0,1,3,2)
np.allclose(gbar_pqrs, -gbar_pqrs.swapaxes(2,3))

相类似的对称性在 $g_{pq}^{rs}$ 中则不存在：

In [ ]:
np.allclose(g_pqrs, g_pqrs.swapaxes(2,3))

同理，我们可以定义 oo-vv 部分的 MO ERI $g_{ij}^{ab}$ 张量：

In [ ]:
gbar_ijab = g_ijab - g_ijab.transpose(0,1,3,2)

### 轨道差张量 $D_{ij}^{ab}$

轨道差张量定义为

\begin{equation}
D_{ij}^{ab} = \varepsilon_i + \varepsilon_j - \varepsilon_a - \varepsilon_b
\end{equation}

下面利用 PySCF 中库 `lib` 的直和功能进行张量的定义．

In [ ]:
# 定义占据轨道能 eo，未占轨道能 ev
eo = e[:nocc]
ev = e[nocc:]
# 定义轨道差张量
D_ijab = lib.direct_sum("i + j - a - b -> ijab", eo, eo, ev, ev)

我们可以验证，上述的直和与下面利用 [boardcasting](https://stackoverflow.com/questions/33848599/performing-outer-addition-with-numpy) 方式构建的张量是等价的：

In [ ]:
np.allclose(eo[:, None, None, None] + eo[None, :, None, None] \
            - ev[None, None, :, None] - ev[None, None, None, :], \
            D_ijab)

### MP2 能量验证

有上述的定义后，我们可以较为轻松地通过下面的公式，计算 MP2 能量：

\begin{equation}
E^\textsf{MP2}_\mathrm{corr} = \frac{(g_{ij}^{ab})^2}{D_{ij}^{ab}} + \frac{1}{2} \frac{(\bar g_{ij}^{ab})^2}{D_{ij}^{ab}}
\end{equation}

In [ ]:
np.allclose((g_ijab ** 2 / D_ijab).sum() + 0.5 * (gbar_ijab ** 2 / D_ijab).sum(), \
            mp2_eng.e_corr)

### MP1 波函数激发系数

一般程序中，MP2 能量通常使用下式计算：

\begin{equation}
E^\textsf{MP2}_\mathrm{corr} = (t_{ij}^{ab})^2 D_{ij}^{ab} + \frac{1}{2} (\bar t_{ij}^{ab})^2 D_{ij}^{ab}
\end{equation}

其中，$t_{ij}^{ab}$ 与 $\bar t_{ij}^{ab}$ 通常称为 MP1 波函数激发系数，定义为

\begin{align}
t_{ij}^{ab} &= g_{ij}^{ab} / D_{ij}^{ab} \\
\bar t_{ij}^{ab} &= \bar g_{ij}^{ab} / D_{ij}^{ab}
\end{align}

在 PySCF 中，激发系数储存于 MP2 计算对象中，非常容易获取．下面我们验证使用 MP1 波函数激发系数的 MP2 相关能结果：

In [ ]:
# 获取激发系数并定义反对称激发系数
t_ijab = mp2_eng.t2
tbar_ijab = t_ijab - t_ijab.swapaxes(2,3)

# 验证激发系数的定义
#-!!!- 这里由于张量较大，以及精度限制，需要将默认的 atol 从 1e-8 降到 1e-7 才能使 np.allclose 给出 True
print(np.allclose(g_ijab / D_ijab, t_ijab, atol=1e-7))

# 验证 MP2 能量
print(np.allclose((t_ijab ** 2 * D_ijab).sum() + 0.5 * (tbar_ijab ** 2 * D_ijab).sum(), \
                  mp2_eng.e_corr))

### RMP1 波函数激发系数对偶正交左矢

在 RMP2 能量与后续计算中，有时使用对偶正交 (Biorthogonal, Helgaker p692) 会大大简化公式复杂性．这里定义

\begin{equation}
T_{ij}^{ab} = 2 t_{ij}^{ab} - t_{ij}^{ba} = t_{ij}^{ab} + \bar t_{ij}^{ab}
\end{equation}

由此，RMP2 能量还可以表示为

\begin{equation}
E^\textsf{MP2}_\mathrm{corr} = T_{ij}^{ab} t_{ij}^{ab} D_{ij}^{ab}
\end{equation}

In [ ]:
T_ijab = 2 * t_ijab - t_ijab.swapaxes(2,3)
np.allclose((T_ijab * t_ijab * D_ijab).sum(), mp2_eng.e_corr)